# 📥 Download Dataset iFood 2019 - Otomatis

Notebook ini akan **otomatis mendownload dan mengekstrak** dataset iFood 2019 ke Google Drive.

### Dataset Info:
| File | Size | Isi |
|------|------|-----|
| Annotations | 3 MB | Labels & class list |
| Train Images | 2.3 GB | 120,216 gambar |
| Val Images | 231 MB | 12,170 gambar |
| Test Images | 548 MB | 28,399 gambar |

**Total: ~3.1 GB**

---

In [ ]:
# ============================================================
# STEP 1: Mount Google Drive
# ============================================================

from google.colab import drive
drive.mount('/content/drive')

import os

# Buat folder struktur
PROJECT_PATH = '/content/drive/MyDrive/AlexNet_iFood2019'
DATASET_PATH = os.path.join(PROJECT_PATH, 'dataset')

os.makedirs(DATASET_PATH, exist_ok=True)
os.makedirs(os.path.join(PROJECT_PATH, 'checkpoints'), exist_ok=True)
os.makedirs(os.path.join(PROJECT_PATH, 'evaluation_results'), exist_ok=True)
os.makedirs(os.path.join(PROJECT_PATH, 'analysis_results'), exist_ok=True)

print(f"✅ Google Drive mounted")
print(f"📁 Dataset akan disimpan di: {DATASET_PATH}")

In [ ]:
# ============================================================
# STEP 2: Download Dataset
# ============================================================

import os
import urllib.request
import tarfile
import time
from tqdm import tqdm

# Dataset URLs dari iFood 2019 Official
DATASET_URLS = {
    'annotations': {
        'url': 'https://food-x.s3.amazonaws.com/annot.tar',
        'filename': 'annot.tar',
        'size': '3 MB',
        'md5': '0c632c543ceed0e70f0eb2db58eda3ab'
    },
    'train': {
        'url': 'https://food-x.s3.amazonaws.com/train.tar',
        'filename': 'train.tar',
        'size': '2.3 GB',
        'md5': '8e56440e365ee852dcb0953a9307e27f'
    },
    'val': {
        'url': 'https://food-x.s3.amazonaws.com/val.tar',
        'filename': 'val.tar',
        'size': '231 MB',
        'md5': 'fa9a4c1eb929835a0fe68734f4868d3b'
    },
    'test': {
        'url': 'https://food-x.s3.amazonaws.com/test.tar',
        'filename': 'test.tar',
        'size': '548 MB',
        'md5': '32479146dd081d38895e46bb93fed58f'
    }
}

class DownloadProgressBar(tqdm):
    def update_to(self, b=1, bsize=1, tsize=None):
        if tsize is not None:
            self.total = tsize
        self.update(b * bsize - self.n)

def download_file(url, output_path, desc):
    """Download file dengan progress bar"""
    with DownloadProgressBar(unit='B', unit_scale=True, miniters=1, desc=desc) as t:
        urllib.request.urlretrieve(url, filename=output_path, reporthook=t.update_to)

# Download semua file
print("📥 Mulai download dataset iFood 2019...")
print("="*60)

download_dir = '/content/downloads'
os.makedirs(download_dir, exist_ok=True)

for name, info in DATASET_URLS.items():
    output_path = os.path.join(download_dir, info['filename'])
    
    # Skip jika sudah ada
    if os.path.exists(output_path):
        print(f"⏭️  {name}: sudah ada, skip download")
        continue
    
    print(f"\n📥 Downloading {name} ({info['size']})...")
    start_time = time.time()
    
    try:
        download_file(info['url'], output_path, info['filename'])
        elapsed = time.time() - start_time
        print(f"✅ {name} selesai dalam {elapsed:.1f} detik")
    except Exception as e:
        print(f"❌ Error downloading {name}: {e}")

print("\n" + "="*60)
print("✅ Semua file berhasil didownload!")

In [ ]:
# ============================================================
# STEP 3: Extract ke Google Drive
# ============================================================

import tarfile
import shutil

download_dir = '/content/downloads'
DATASET_PATH = '/content/drive/MyDrive/AlexNet_iFood2019/dataset'

print("📦 Mulai ekstraksi ke Google Drive...")
print("="*60)

# 1. Extract annotations
print("\n📦 Extracting annotations...")
annot_tar = os.path.join(download_dir, 'annot.tar')
if os.path.exists(annot_tar):
    with tarfile.open(annot_tar, 'r') as tar:
        tar.extractall(DATASET_PATH)
    print("✅ Annotations extracted")

# 2. Extract train images
print("\n📦 Extracting train images (ini akan memakan waktu ~10-15 menit)...")
train_tar = os.path.join(download_dir, 'train.tar')
train_dir = os.path.join(DATASET_PATH, 'train_images')
if os.path.exists(train_tar):
    os.makedirs(train_dir, exist_ok=True)
    with tarfile.open(train_tar, 'r') as tar:
        # Extract to temp then move
        tar.extractall('/content/temp_train')
    # Move images to correct folder
    src_dir = '/content/temp_train/train_set'
    if os.path.exists(src_dir):
        for f in os.listdir(src_dir):
            shutil.move(os.path.join(src_dir, f), train_dir)
        shutil.rmtree('/content/temp_train')
    print(f"✅ Train images extracted: {len(os.listdir(train_dir))} files")

# 3. Extract val images
print("\n📦 Extracting validation images...")
val_tar = os.path.join(download_dir, 'val.tar')
val_dir = os.path.join(DATASET_PATH, 'val_images')
if os.path.exists(val_tar):
    os.makedirs(val_dir, exist_ok=True)
    with tarfile.open(val_tar, 'r') as tar:
        tar.extractall('/content/temp_val')
    src_dir = '/content/temp_val/val_set'
    if os.path.exists(src_dir):
        for f in os.listdir(src_dir):
            shutil.move(os.path.join(src_dir, f), val_dir)
        shutil.rmtree('/content/temp_val')
    print(f"✅ Val images extracted: {len(os.listdir(val_dir))} files")

# 4. Extract test images
print("\n📦 Extracting test images...")
test_tar = os.path.join(download_dir, 'test.tar')
test_dir = os.path.join(DATASET_PATH, 'test_images')
if os.path.exists(test_tar):
    os.makedirs(test_dir, exist_ok=True)
    with tarfile.open(test_tar, 'r') as tar:
        tar.extractall('/content/temp_test')
    src_dir = '/content/temp_test/test_set'
    if os.path.exists(src_dir):
        for f in os.listdir(src_dir):
            shutil.move(os.path.join(src_dir, f), test_dir)
        shutil.rmtree('/content/temp_test')
    print(f"✅ Test images extracted: {len(os.listdir(test_dir))} files")

print("\n" + "="*60)
print("✅ Ekstraksi selesai!")

In [ ]:
# ============================================================
# STEP 4: Verifikasi Dataset
# ============================================================

import os

DATASET_PATH = '/content/drive/MyDrive/AlexNet_iFood2019/dataset'

print("🔍 Verifikasi dataset...")
print("="*60)

# Check files
required_items = {
    'class_list.txt': 'file',
    'train_info.csv': 'file',
    'val_info.csv': 'file',
    'test_info.csv': 'file',
    'train_images': 'dir',
    'val_images': 'dir',
    'test_images': 'dir'
}

all_ok = True
for item, item_type in required_items.items():
    path = os.path.join(DATASET_PATH, item)
    
    if item_type == 'file':
        exists = os.path.isfile(path)
    else:
        exists = os.path.isdir(path)
    
    if exists:
        if item_type == 'dir':
            count = len(os.listdir(path))
            print(f"✅ {item}: {count:,} files")
        else:
            print(f"✅ {item}")
    else:
        print(f"❌ {item}: TIDAK DITEMUKAN")
        all_ok = False

print("\n" + "="*60)
if all_ok:
    print("🎉 DATASET SIAP DIGUNAKAN!")
    print(f"\nLokasi: {DATASET_PATH}")
else:
    print("⚠️  Ada file yang hilang, coba jalankan ulang dari Step 2")

In [ ]:
{
  "cells": [
    {
      "cell_type": "markdown",
      "metadata": {},
      "source": [
        "# 📥 Download Dataset iFood 2019 - Otomatis\n",
        "\n",
        "Notebook ini akan **otomatis mendownload dan mengekstrak** dataset iFood 2019 ke Google Drive.\n",
        "\n",
        "### Dataset Info:\n",
        "| File | Size | Isi |\n",
        "|------|------|-----|\n",
        "| Annotations | 3 MB | Labels & class list |\n",
        "| Train Images | 2.3 GB | 120,216 gambar |\n",
        "| Val Images | 231 MB | 12,170 gambar |\n",
        "| Test Images | 548 MB | 28,399 gambar |\n",
        "\n",
        "**Total: ~3.1 GB**\n",
        "\n",
        "---"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# ============================================================\n",
        "# STEP 1: Mount Google Drive\n",
        "# ============================================================\n",
        "\n",
        "from google.colab import drive\n",
        "drive.mount('/content/drive')\n",
        "\n",
        "import os\n",
        "\n",
        "# Buat folder struktur\n",
        "PROJECT_PATH = '/content/drive/MyDrive/AlexNet_iFood2019'\n",
        "DATASET_PATH = os.path.join(PROJECT_PATH, 'dataset')\n",
        "\n",
        "os.makedirs(DATASET_PATH, exist_ok=True)\n",
        "os.makedirs(os.path.join(PROJECT_PATH, 'checkpoints'), exist_ok=True)\n",
        "os.makedirs(os.path.join(PROJECT_PATH, 'evaluation_results'), exist_ok=True)\n",
        "os.makedirs(os.path.join(PROJECT_PATH, 'analysis_results'), exist_ok=True)\n",
        "\n",
        "print(f\"✅ Google Drive mounted\")\n",
        "print(f\"📁 Dataset akan disimpan di: {DATASET_PATH}\")"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# ============================================================\n",
        "# STEP 2: Download Dataset\n",
        "# ============================================================\n",
        "\n",
        "import os\n",
        "import urllib.request\n",
        "import tarfile\n",
        "import time\n",
        "from tqdm import tqdm\n",
        "\n",
        "# Dataset URLs dari iFood 2019 Official\n",
        "DATASET_URLS = {\n",
        "    'annotations': {\n",
        "        'url': 'https://food-x.s3.amazonaws.com/annot.tar',\n",
        "        'filename': 'annot.tar',\n",
        "        'size': '3 MB',\n",
        "        'md5': '0c632c543ceed0e70f0eb2db58eda3ab'\n",
        "    },\n",
        "    'train': {\n",
        "        'url': 'https://food-x.s3.amazonaws.com/train.tar',\n",
        "        'filename': 'train.tar',\n",
        "        'size': '2.3 GB',\n",
        "        'md5': '8e56440e365ee852dcb0953a9307e27f'\n",
        "    },\n",
        "    'val': {\n",
        "        'url': 'https://food-x.s3.amazonaws.com/val.tar',\n",
        "        'filename': 'val.tar',\n",
        "        'size': '231 MB',\n",
        "        'md5': 'fa9a4c1eb929835a0fe68734f4868d3b'\n",
        "    },\n",
        "    'test': {\n",
        "        'url': 'https://food-x.s3.amazonaws.com/test.tar',\n",
        "        'filename': 'test.tar',\n",
        "        'size': '548 MB',\n",
        "        'md5': '32479146dd081d38895e46bb93fed58f'\n",
        "    }\n",
        "}\n",
        "\n",
        "class DownloadProgressBar(tqdm):\n",
        "    def update_to(self, b=1, bsize=1, tsize=None):\n",
        "        if tsize is not None:\n",
        "            self.total = tsize\n",
        "        self.update(b * bsize - self.n)\n",
        "\n",
        "def download_file(url, output_path, desc):\n",
        "    \"\"\"Download file dengan progress bar\"\"\"\n",
        "    with DownloadProgressBar(unit='B', unit_scale=True, miniters=1, desc=desc) as t:\n",
        "        urllib.request.urlretrieve(url, filename=output_path, reporthook=t.update_to)\n",
        "\n",
        "# Download semua file\n",
        "print(\"📥 Mulai download dataset iFood 2019...\")\n",
        "print(\"=\"*60)\n",
        "\n",
        "download_dir = '/content/downloads'\n",
        "os.makedirs(download_dir, exist_ok=True)\n",
        "\n",
        "for name, info in DATASET_URLS.items():\n",
        "    output_path = os.path.join(download_dir, info['filename'])\n",
        "    \n",
        "    # Skip jika sudah ada\n",
        "    if os.path.exists(output_path):\n",
        "        print(f\"⏭️  {name}: sudah ada, skip download\")\n",
        "        continue\n",
        "    \n",
        "    print(f\"\\n📥 Downloading {name} ({info['size']})...\")\n",
        "    start_time = time.time()\n",
        "    \n",
        "    try:\n",
        "        download_file(info['url'], output_path, info['filename'])\n",
        "        elapsed = time.time() - start_time\n",
        "        print(f\"✅ {name} selesai dalam {elapsed:.1f} detik\")\n",
        "    except Exception as e:\n",
        "        print(f\"❌ Error downloading {name}: {e}\")\n",
        "\n",
        "print(\"\\n\" + \"=\"*60)\n",
        "print(\"✅ Semua file berhasil didownload!\")"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# ============================================================\n",
        "# STEP 3: Extract ke Google Drive (FIXED - handle existing files)\n",
        "# ============================================================\n",
        "\n",
        "import tarfile\n",
        "import shutil\n",
        "import os\n",
        "from tqdm import tqdm\n",
        "\n",
        "download_dir = '/content/downloads'\n",
        "DATASET_PATH = '/content/drive/MyDrive/AlexNet_iFood2019/dataset'\n",
        "\n",
        "def safe_extract(tar_path, extract_to, filter_arg='data'):\n",
        "    \"\"\"Extract tar file dengan handling untuk Python 3.12+\"\"\"\n",
        "    with tarfile.open(tar_path, 'r') as tar:\n",
        "        # Gunakan filter untuk Python 3.12+ (menghilangkan warning)\n",
        "        try:\n",
        "            tar.extractall(extract_to, filter=filter_arg)\n",
        "        except TypeError:\n",
        "            # Fallback untuk Python versi lama\n",
        "            tar.extractall(extract_to)\n",
        "\n",
        "def move_files_safe(src_dir, dest_dir):\n",
        "    \"\"\"Move files dengan skip jika sudah ada\"\"\"\n",
        "    if not os.path.exists(src_dir):\n",
        "        return 0\n",
        "    \n",
        "    files = os.listdir(src_dir)\n",
        "    moved = 0\n",
        "    skipped = 0\n",
        "    \n",
        "    for f in tqdm(files, desc=\"Moving files\"):\n",
        "        src_path = os.path.join(src_dir, f)\n",
        "        dest_path = os.path.join(dest_dir, f)\n",
        "        \n",
        "        if os.path.exists(dest_path):\n",
        "            # File sudah ada, skip\n",
        "            skipped += 1\n",
        "            continue\n",
        "        \n",
        "        try:\n",
        "            shutil.move(src_path, dest_path)\n",
        "            moved += 1\n",
        "        except Exception as e:\n",
        "            print(f\"Warning: Could not move {f}: {e}\")\n",
        "    \n",
        "    if skipped > 0:\n",
        "        print(f\"   ℹ️  Skipped {skipped} existing files\")\n",
        "    \n",
        "    return moved + skipped\n",
        "\n",
        "print(\"📦 Mulai ekstraksi ke Google Drive...\")\n",
        "print(\"=\"*60)\n",
        "\n",
        "# 1. Extract annotations\n",
        "print(\"\\n📦 Extracting annotations...\")\n",
        "annot_tar = os.path.join(download_dir, 'annot.tar')\n",
        "if os.path.exists(annot_tar):\n",
        "    safe_extract(annot_tar, DATASET_PATH)\n",
        "    print(\"✅ Annotations extracted\")\n",
        "else:\n",
        "    print(\"⏭️  Annotations tar not found, skipping\")\n",
        "\n",
        "# 2. Extract train images\n",
        "print(\"\\n📦 Extracting train images (ini akan memakan waktu ~10-15 menit)...\")\n",
        "train_tar = os.path.join(download_dir, 'train.tar')\n",
        "train_dir = os.path.join(DATASET_PATH, 'train_images')\n",
        "temp_train = '/content/temp_train'\n",
        "\n",
        "if os.path.exists(train_tar):\n",
        "    os.makedirs(train_dir, exist_ok=True)\n",
        "    \n",
        "    # Clean temp folder if exists\n",
        "    if os.path.exists(temp_train):\n",
        "        shutil.rmtree(temp_train)\n",
        "    \n",
        "    print(\"   Extracting to temp...\")\n",
        "    safe_extract(train_tar, temp_train)\n",
        "    \n",
        "    # Find the extracted folder\n",
        "    src_dir = os.path.join(temp_train, 'train_set')\n",
        "    if not os.path.exists(src_dir):\n",
        "        # Try alternative path\n",
        "        subdirs = os.listdir(temp_train)\n",
        "        if subdirs:\n",
        "            src_dir = os.path.join(temp_train, subdirs[0])\n",
        "    \n",
        "    if os.path.exists(src_dir):\n",
        "        print(\"   Moving to Drive...\")\n",
        "        count = move_files_safe(src_dir, train_dir)\n",
        "        shutil.rmtree(temp_train, ignore_errors=True)\n",
        "        print(f\"✅ Train images: {len(os.listdir(train_dir))} files\")\n",
        "    else:\n",
        "        print(\"❌ Could not find train_set folder\")\n",
        "else:\n",
        "    print(f\"⏭️  Train tar not found, checking existing: {len(os.listdir(train_dir)) if os.path.exists(train_dir) else 0} files\")\n",
        "\n",
        "# 3. Extract val images\n",
        "print(\"\\n📦 Extracting validation images...\")\n",
        "val_tar = os.path.join(download_dir, 'val.tar')\n",
        "val_dir = os.path.join(DATASET_PATH, 'val_images')\n",
        "temp_val = '/content/temp_val'\n",
        "\n",
        "if os.path.exists(val_tar):\n",
        "    os.makedirs(val_dir, exist_ok=True)\n",
        "    \n",
        "    if os.path.exists(temp_val):\n",
        "        shutil.rmtree(temp_val)\n",
        "    \n",
        "    safe_extract(val_tar, temp_val)\n",
        "    \n",
        "    src_dir = os.path.join(temp_val, 'val_set')\n",
        "    if not os.path.exists(src_dir):\n",
        "        subdirs = os.listdir(temp_val)\n",
        "        if subdirs:\n",
        "            src_dir = os.path.join(temp_val, subdirs[0])\n",
        "    \n",
        "    if os.path.exists(src_dir):\n",
        "        count = move_files_safe(src_dir, val_dir)\n",
        "        shutil.rmtree(temp_val, ignore_errors=True)\n",
        "        print(f\"✅ Val images: {len(os.listdir(val_dir))} files\")\n",
        "else:\n",
        "    print(f\"⏭️  Val tar not found, checking existing: {len(os.listdir(val_dir)) if os.path.exists(val_dir) else 0} files\")\n",
        "\n",
        "# 4. Extract test images\n",
        "print(\"\\n📦 Extracting test images...\")\n",
        "test_tar = os.path.join(download_dir, 'test.tar')\n",
        "test_dir = os.path.join(DATASET_PATH, 'test_images')\n",
        "temp_test = '/content/temp_test'\n",
        "\n",
        "if os.path.exists(test_tar):\n",
        "    os.makedirs(test_dir, exist_ok=True)\n",
        "    \n",
        "    if os.path.exists(temp_test):\n",
        "        shutil.rmtree(temp_test)\n",
        "    \n",
        "    safe_extract(test_tar, temp_test)\n",
        "    \n",
        "    src_dir = os.path.join(temp_test, 'test_set')\n",
        "    if not os.path.exists(src_dir):\n",
        "        subdirs = os.listdir(temp_test)\n",
        "        if subdirs:\n",
        "            src_dir = os.path.join(temp_test, subdirs[0])\n",
        "    \n",
        "    if os.path.exists(src_dir):\n",
        "        count = move_files_safe(src_dir, test_dir)\n",
        "        shutil.rmtree(temp_test, ignore_errors=True)\n",
        "        print(f\"✅ Test images: {len(os.listdir(test_dir))} files\")\n",
        "else:\n",
        "    print(f\"⏭️  Test tar not found, checking existing: {len(os.listdir(test_dir)) if os.path.exists(test_dir) else 0} files\")\n",
        "\n",
        "print(\"\\n\" + \"=\"*60)\n",
        "print(\"✅ Ekstraksi selesai!\")"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# ============================================================\n",
        "# STEP 4: Verifikasi Dataset\n",
        "# ============================================================\n",
        "\n",
        "import os\n",
        "\n",
        "DATASET_PATH = '/content/drive/MyDrive/AlexNet_iFood2019/dataset'\n",
        "\n",
        "print(\"🔍 Verifikasi dataset...\")\n",
        "print(\"=\"*60)\n",
        "\n",
        "# Check files\n",
        "required_items = {\n",
        "    'class_list.txt': 'file',\n",
        "    'train_info.csv': 'file',\n",
        "    'val_info.csv': 'file',\n",
        "    'test_info.csv': 'file',\n",
        "    'train_images': 'dir',\n",
        "    'val_images': 'dir',\n",
        "    'test_images': 'dir'\n",
        "}\n",
        "\n",
        "all_ok = True\n",
        "for item, item_type in required_items.items():\n",
        "    path = os.path.join(DATASET_PATH, item)\n",
        "    \n",
        "    if item_type == 'file':\n",
        "        exists = os.path.isfile(path)\n",
        "    else:\n",
        "        exists = os.path.isdir(path)\n",
        "    \n",
        "    if exists:\n",
        "        if item_type == 'dir':\n",
        "            count = len(os.listdir(path))\n",
        "            print(f\"✅ {item}: {count:,} files\")\n",
        "        else:\n",
        "            print(f\"✅ {item}\")\n",
        "    else:\n",
        "        print(f\"❌ {item}: TIDAK DITEMUKAN\")\n",
        "        all_ok = False\n",
        "\n",
        "print(\"\\n\" + \"=\"*60)\n",
        "if all_ok:\n",
        "    print(\"🎉 DATASET SIAP DIGUNAKAN!\")\n",
        "    print(f\"\\nLokasi: {DATASET_PATH}\")\n",
        "else:\n",
        "    print(\"⚠️  Ada file yang hilang, coba jalankan ulang dari Step 2\")"
      ]
    },
    {
      "cell_type": "code",
      "execution_count": null,
      "metadata": {},
      "outputs": [],
      "source": [
        "# ============================================================\n",
        "# STEP 5: Cleanup (Hapus file tar untuk hemat storage)\n",
        "# ============================================================\n",
        "\n",
        "import shutil\n",
        "import os\n",
        "\n",
        "download_dir = '/content/downloads'\n",
        "\n",
        "if os.path.exists(download_dir):\n",
        "    size_before = sum(os.path.getsize(os.path.join(download_dir, f)) \n",
        "                      for f in os.listdir(download_dir) \n",
        "                      if os.path.isfile(os.path.join(download_dir, f)))\n",
        "    \n",
        "    shutil.rmtree(download_dir)\n",
        "    print(f\"🗑️  Deleted download cache: {size_before / 1e9:.2f} GB freed\")\n",
        "else:\n",
        "    print(\"✅ No cache to clean\")\n",
        "\n",
        "# Cleanup temp folders juga\n",
        "for temp in ['/content/temp_train', '/content/temp_val', '/content/temp_test']:\n",
        "    if os.path.exists(temp):\n",
        "        shutil.rmtree(temp, ignore_errors=True)\n",
        "\n",
        "print(\"\\n\" + \"=\"*60)\n",
        "print(\"🎉 SELESAI!\")\n",
        "print(\"=\"*60)\n",
        "print(\"\\nDataset sudah tersimpan di Google Drive.\")\n",
        "print(\"Anda bisa menutup notebook ini dan lanjut ke training.\")\n",
        "print(\"\\nNotebook selanjutnya:\")\n",
        "print(\"  - train_member1_baseline.ipynb (Member 1)\")\n",
        "print(\"  - train_member2_mod1.ipynb (Member 2)\")\n",
        "print(\"  - train_member3_mod2.ipynb (Member 3)\")\n",
        "print(\"  - train_member4_combined.ipynb (Member 4)\")"
      ]
    }
  ],
  "metadata": {
    "kernelspec": {
      "display_name": "Python 3",
      "language": "python",
      "name": "python3"
    },
    "accelerator": "GPU"
  },
  "nbformat": 4,
  "nbformat_minor": 4
}